# SWD 提取points 边界

In [33]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import re
import json
import glob
from pathlib import Path
from typing import Any, Dict, Iterable, List, Set, Tuple
from collections import defaultdict

# ========= 配置（按需修改） =========
JPG_DIR = Path("/workspace/models/runs_yolov11_det/cropped_objects_v6/swd")

# v7 结构：run_v7/*/raw_data_sliced_merge/filtered_annotations_v2..v9.json
RUN_ROOTS = [Path("/workspace/models/SAHI/run_v7")]

# v4/5/6 结构：run_vX/raw_data_sliced_merge/filtered_annotations.json
RUN_ROOTS2 = [
    Path("/workspace/models/SAHI/run_v4"),
    Path("/workspace/models/SAHI/run_v5"),
    Path("/workspace/models/SAHI/run_v6"),
]

# 输出文件
OUT_JSONL = Path("matched_points.jsonl")
OUT_UNMATCHED = Path("unmatched_uuids.txt")

# ========= 工具函数 =========

UUID_RE = re.compile(r"_uuid_([0-9a-fA-F-]{36})(?:\.[jJ][pP][eE]?[gG])?$")

def extract_uuid_from_filename(name: str) -> str | None:
    m = UUID_RE.search(name)
    return m.group(1).lower() if m else None

def find_all_jpg_uuids(jpg_dir: Path) -> Tuple[Set[str], List[Tuple[str, str]]]:
    uuids: Set[str] = set()
    pairs: List[Tuple[str, str]] = []
    for p in sorted(jpg_dir.glob("*.jpg")):
        uid = extract_uuid_from_filename(p.name)
        if uid:
            uuids.add(uid)
            pairs.append((uid, p.stem))
    return uuids, pairs

def iter_filtered_json_paths_v7(roots: List[Path]) -> Iterable[Path]:
    """
    run_v7：run_v7/*/raw_data_sliced_merge/filtered_annotations_v2..v9.json
    """
    for root in roots:
        pattern = str(root / "*" / "raw_data_sliced_merge" / "filtered_annotations_v[2-9].json")
        for p in glob.iglob(pattern):
            yield Path(p)

def iter_filtered_json_paths_v456(roots: List[Path]) -> Iterable[Path]:
    """
    run_v4/5/6：run_vX/raw_data_sliced_merge/filtered_annotations.json
    """
    for root in roots:
        pattern = str(root / "raw_data_sliced_merge" / "merged_annotations.json")
        for p in glob.iglob(pattern):
            yield Path(p)

def walk_json(obj: Any) -> Iterable[Dict[str, Any]]:
    if isinstance(obj, dict):
        yield obj
        for v in obj.values():
            yield from walk_json(v)
    elif isinstance(obj, list):
        for it in obj:
            yield from walk_json(it)

# ========= 主逻辑 =========

def main():
    # 1) 收集所有 jpg 的 uuid
    uuid_set, uuid_pairs = find_all_jpg_uuids(JPG_DIR)
    if not uuid_set:
        print(f"[WARN] 没在 {JPG_DIR} 找到任何符合模式的 jpg/uuid。")
        return
    print(f"[INFO] 从 JPG 提取到 {len(uuid_set)} 个唯一 UUID。")

    # 2) 汇总 JSON 路径
    json_paths = list(iter_filtered_json_paths_v7(RUN_ROOTS))
    json_paths.extend(iter_filtered_json_paths_v456(RUN_ROOTS2))
    if not json_paths:
        print(f"[WARN] 没找到任何匹配的 JSON（v7: v2~v9；v4/5/6: filtered_annotations.json）。")
        return
    print(f"[INFO] 找到 {len(json_paths)} 个 JSON 文件可供检索。")

    # 3) 聚合：uuid -> 去重后的 points 列表（同一 uuid 保持唯一行）
    agg_items: Dict[str, List[Dict[str, Any]]] = defaultdict(list)
    seen_points_keys: Dict[str, Set[str]] = defaultdict(set)  # uuid -> {serialized points}

    def _serialize_points(pts: Any) -> str:
        # 将 points 规范序列化用于去重
        try:
            return json.dumps(pts, ensure_ascii=False, sort_keys=True)
        except Exception:
            return str(pts)

    # 遍历 JSON
    for jpath in json_paths:
        try:
            with jpath.open("r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print(f"[ERROR] 读取 JSON 失败：{jpath} -> {e}")
            continue

        for d in walk_json(data):
            key = "uuid" if "uuid" in d else ("UUID" if "UUID" in d else None)
            if not key:
                continue
            uid = str(d.get(key, "")).lower()
            if uid in uuid_set:
                points = d.get("points", None)
                original_name = d.get("original_name", None)
                pkey = _serialize_points(points)
                if pkey not in seen_points_keys[uid]:
                    seen_points_keys[uid].add(pkey)
                    agg_items[uid].append({"json_path": str(jpath), "original_name": str(original_name), "points": points})

    # 4) 写出结果（每个 uuid 一行）
    matched_uuids = set(agg_items.keys())
    with OUT_JSONL.open("w", encoding="utf-8") as fout:
        for uid, items in agg_items.items():
            rec = {
                "uuid": uid,
                "matches": items  # 去重后的 {json_path, points} 列表
            }
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")

    # 5) 未命中统计
    unmatched = [u for u, _ in uuid_pairs if u not in matched_uuids]
    with OUT_UNMATCHED.open("w", encoding="utf-8") as f:
        for u in unmatched:
            f.write(u + "\n")

    print(f"[DONE] 命中 {sum(len(v) for v in agg_items.values())} 条记录，涉及 {len(matched_uuids)} 个 UUID。")
    print(f"[DONE] 结果已写入：{OUT_JSONL.resolve()}（每行一个 uuid，内部 points 去重）")
    print(f"[DONE] 未命中的 UUID 共 {len(unmatched)} 个，已写入：{OUT_UNMATCHED.resolve()}")

if __name__ == "__main__":
    main()


[INFO] 从 JPG 提取到 849 个唯一 UUID。
[INFO] 找到 227 个 JSON 文件可供检索。


[DONE] 命中 849 条记录，涉及 849 个 UUID。
[DONE] 结果已写入：/workspace/models/runs_yolov11_det/matched_points.jsonl（每行一个 uuid，内部 points 去重）
[DONE] 未命中的 UUID 共 0 个，已写入：/workspace/models/runs_yolov11_det/unmatched_uuids.txt


# mayswd 提取points 边界

In [30]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import re
import json
import glob
from pathlib import Path
from typing import Any, Dict, Iterable, List, Set, Tuple
from collections import defaultdict

# ========= 配置（按需修改） =========
JPG_DIR = Path("/workspace/models/runs_yolov11_det/cropped_objects_v6/mayswd")

# v7 结构：run_v7/*/raw_data_sliced_merge/filtered_annotations_v2..v9.json
RUN_ROOTS = [Path("/workspace/models/SAHI/run_v7")]

# v4/5/6 结构：run_vX/raw_data_sliced_merge/filtered_annotations.json
RUN_ROOTS2 = [
    Path("/workspace/models/SAHI/run_v3"),
    Path("/workspace/models/SAHI/run_v4"),
    Path("/workspace/models/SAHI/run_v5"),
    Path("/workspace/models/SAHI/run_v6"),
]

# 输出文件
OUT_JSONL = Path("matched_points_mayswd.jsonl")
OUT_UNMATCHED = Path("unmatched_uuids_mayswd.txt")

# ========= 工具函数 =========

UUID_RE = re.compile(r"_uuid_([0-9a-fA-F-]{36})(?:\.[jJ][pP][eE]?[gG])?$")

def extract_uuid_from_filename(name: str) -> str | None:
    m = UUID_RE.search(name)
    return m.group(1).lower() if m else None

def find_all_jpg_uuids(jpg_dir: Path) -> Tuple[Set[str], List[Tuple[str, str]]]:
    uuids: Set[str] = set()
    pairs: List[Tuple[str, str]] = []
    for p in sorted(jpg_dir.glob("*.jpg")):
        uid = extract_uuid_from_filename(p.name)
        if uid:
            uuids.add(uid)
            pairs.append((uid, p.stem))
    return uuids, pairs

def iter_filtered_json_paths_v7(roots: List[Path]) -> Iterable[Path]:
    """
    run_v7：run_v7/*/raw_data_sliced_merge/filtered_annotations_v2..v9.json
    """
    for root in roots:
        pattern = str(root / "*" / "raw_data_sliced_merge" / "filtered_annotations_v[2-9].json")
        for p in glob.iglob(pattern):
            yield Path(p)

def iter_filtered_json_paths_v456(roots: List[Path]) -> Iterable[Path]:
    """
    run_v4/5/6：run_vX/raw_data_sliced_merge/filtered_annotations.json
    """
    for root in roots:
        pattern = str(root / "raw_data_sliced_merge" / "merged_annotations.json")
        for p in glob.iglob(pattern):
            yield Path(p)

def walk_json(obj: Any) -> Iterable[Dict[str, Any]]:
    if isinstance(obj, dict):
        yield obj
        for v in obj.values():
            yield from walk_json(v)
    elif isinstance(obj, list):
        for it in obj:
            yield from walk_json(it)

# ========= 主逻辑 =========

def main():
    # 1) 收集所有 jpg 的 uuid
    uuid_set, uuid_pairs = find_all_jpg_uuids(JPG_DIR)
    if not uuid_set:
        print(f"[WARN] 没在 {JPG_DIR} 找到任何符合模式的 jpg/uuid。")
        return
    print(f"[INFO] 从 JPG 提取到 {len(uuid_set)} 个唯一 UUID。")

    # 2) 汇总 JSON 路径
    json_paths = list(iter_filtered_json_paths_v7(RUN_ROOTS))
    json_paths.extend(iter_filtered_json_paths_v456(RUN_ROOTS2))
    if not json_paths:
        print(f"[WARN] 没找到任何匹配的 JSON（v7: v2~v9；v4/5/6: filtered_annotations.json）。")
        return
    print(f"[INFO] 找到 {len(json_paths)} 个 JSON 文件可供检索。")

    # 3) 聚合：uuid -> 去重后的 points 列表（同一 uuid 保持唯一行）
    agg_items: Dict[str, List[Dict[str, Any]]] = defaultdict(list)
    seen_points_keys: Dict[str, Set[str]] = defaultdict(set)  # uuid -> {serialized points}

    def _serialize_points(pts: Any) -> str:
        # 将 points 规范序列化用于去重
        try:
            return json.dumps(pts, ensure_ascii=False, sort_keys=True)
        except Exception:
            return str(pts)

    # 遍历 JSON
    for jpath in json_paths:
        try:
            with jpath.open("r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print(f"[ERROR] 读取 JSON 失败：{jpath} -> {e}")
            continue

        for d in walk_json(data):
            key = "uuid" if "uuid" in d else ("UUID" if "UUID" in d else None)
            if not key:
                continue
            uid = str(d.get(key, "")).lower()
            if uid in uuid_set:
                points = d.get("points", None)
                original_name = d.get("original_name", None)
                pkey = _serialize_points(points)
                if pkey not in seen_points_keys[uid]:
                    seen_points_keys[uid].add(pkey)
                    agg_items[uid].append({"json_path": str(jpath), "original_name": str(original_name), "points": points})

    # 4) 写出结果（每个 uuid 一行）
    matched_uuids = set(agg_items.keys())
    with OUT_JSONL.open("w", encoding="utf-8") as fout:
        for uid, items in agg_items.items():
            rec = {
                "uuid": uid,
                "matches": items  # 去重后的 {json_path, points} 列表
            }
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")

    # 5) 未命中统计
    unmatched = [u for u, _ in uuid_pairs if u not in matched_uuids]
    with OUT_UNMATCHED.open("w", encoding="utf-8") as f:
        for u in unmatched:
            f.write(u + "\n")

    print(f"[DONE] 命中 {sum(len(v) for v in agg_items.values())} 条记录，涉及 {len(matched_uuids)} 个 UUID。")
    print(f"[DONE] 结果已写入：{OUT_JSONL.resolve()}（每行一个 uuid，内部 points 去重）")
    print(f"[DONE] 未命中的 UUID 共 {len(unmatched)} 个，已写入：{OUT_UNMATCHED.resolve()}")

if __name__ == "__main__":
    main()


[INFO] 从 JPG 提取到 38 个唯一 UUID。
[INFO] 找到 228 个 JSON 文件可供检索。
[DONE] 命中 38 条记录，涉及 38 个 UUID。
[DONE] 结果已写入：/workspace/models/runs_yolov11_det/matched_points_mayswd.jsonl（每行一个 uuid，内部 points 去重）
[DONE] 未命中的 UUID 共 0 个，已写入：/workspace/models/runs_yolov11_det/unmatched_uuids_mayswd.txt


# 抠图

In [59]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
import cv2
import glob
import numpy as np
from pathlib import Path
from typing import List, Tuple, Any, Optional, Dict

# ========= 配置 =========
# JSONL_PATH   = Path("matched_points_mayswd.jsonl")
JSONL_PATH   = Path("matched_points.jsonl")
OUT_DIR_RECT = Path("/workspace/models/runs_yolov11_det/crops_by_points/rect")  # 正方形裁剪（取长边）
OUT_DIR_MASK = Path("/workspace/models/runs_yolov11_det/crops_by_points/mask")  # 多边形抠图（透明）
SAVE_BBOX    = True
SAVE_MASK    = True

# ========= 工具 =========
IMG_EXTS = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]

def to_xy_list(points: Any) -> List[Tuple[float, float]]:
    """将 points 统一为 [(x,y), ...]，兼容 [[x,y], ...] 或 [{'x':..,'y':..}, ...]"""
    if points is None:
        return []
    xy = []
    if isinstance(points, (list, tuple)):
        for pt in points:
            if isinstance(pt, (list, tuple)) and len(pt) >= 2:
                x, y = float(pt[0]), float(pt[1])
                xy.append((x, y))
            elif isinstance(pt, dict) and 'x' in pt and 'y' in pt:
                xy.append((float(pt['x']), float(pt['y'])))
    return xy

def clamp_bbox(xmin, ymin, xmax, ymax, w, h):
    xmin = max(0, min(int(np.floor(xmin)), w-1))
    ymin = max(0, min(int(np.floor(ymin)), h-1))
    xmax = max(0, min(int(np.ceil (xmax)), w-1))
    ymax = max(0, min(int(np.ceil (ymax)), h-1))
    if xmax < xmin: xmax = xmin
    if ymax < ymin: ymax = ymin
    return xmin, ymin, xmax, ymax

def square_bbox_from_points(xy: List[Tuple[float, float]], w: int, h: int) -> Tuple[int,int,int,int]:
    """
    基于点集的外接矩形，扩为以中心为基准的正方形（边长=长边）。
    若越界，则整体平移窗口以尽量保持完整正方形；最后 clamp 到图内。
    返回：xmin, ymin, xmax, ymax
    """
    xs = [p[0] for p in xy]; ys = [p[1] for p in xy]
    xmin0, ymin0, xmax0, ymax0 = min(xs), min(ys), max(xs), max(ys)
    bw = xmax0 - xmin0 + 1
    bh = ymax0 - ymin0 + 1
    side = int(np.ceil(max(bw, bh)))  # 取长边并向上取整

    # 以中心为基准构建正方形窗口
    cx = (xmin0 + xmax0) / 2.0
    cy = (ymin0 + ymax0) / 2.0
    xmin = int(np.floor(cx - side / 2.0))
    ymin = int(np.floor(cy - side / 2.0))
    xmax = xmin + side - 1
    ymax = ymin + side - 1

    # 越界则整体平移
    if xmin < 0:
        shift = -xmin
        xmin += shift; xmax += shift
    if ymin < 0:
        shift = -ymin
        ymin += shift; ymax += shift
    if xmax >= w:
        shift = xmax - (w - 1)
        xmin -= shift; xmax -= shift
    if ymax >= h:
        shift = ymax - (h - 1)
        ymin -= shift; ymax -= shift

    # 最终 clamp
    xmin, ymin, xmax, ymax = clamp_bbox(xmin, ymin, xmax, ymax, w, h)

    # 保底再对齐为正方形（在边界内微调）
    cur_w = xmax - xmin + 1
    cur_h = ymax - ymin + 1
    if cur_w != cur_h:
        side = min(max(cur_w, cur_h), min(w, h))
        cx = (xmin + xmax) / 2.0
        cy = (ymin + ymax) / 2.0
        xmin = int(np.floor(cx - side / 2.0))
        ymin = int(np.floor(cy - side / 2.0))
        xmax = xmin + side - 1
        ymax = ymin + side - 1
        xmin, ymin, xmax, ymax = clamp_bbox(xmin, ymin, xmax, ymax, w, h)

    return xmin, ymin, xmax, ymax

def crop_bbox_square(img: np.ndarray, xy: List[Tuple[float, float]]) -> Tuple[np.ndarray, Tuple[int,int,int,int]]:
    """按长边扩成正方形的裁切，返回 (crop, (xmin,ymin,xmax,ymax))."""
    h, w = img.shape[:2]
    xmin, ymin, xmax, ymax = square_bbox_from_points(xy, w, h)
    return img[ymin:ymax+1, xmin:xmax+1].copy(), (xmin, ymin, xmax, ymax)

def crop_polygon_mask(img: np.ndarray, xy: List[Tuple[float, float]]) -> Tuple[np.ndarray, Tuple[int,int,int,int]]:
    """按多边形外接矩形裁切并制作透明 Mask，返回 (rgba_crop, (xmin,ymin,xmax,ymax))."""
    xs = [p[0] for p in xy]; ys = [p[1] for p in xy]
    h, w = img.shape[:2]
    xmin, ymin, xmax, ymax = clamp_bbox(min(xs), min(ys), max(xs), max(ys), w, h)

    crop = img[ymin:ymax+1, xmin:xmax+1].copy()
    ch, cw = crop.shape[:2]

    pts = np.array([[p[0]-xmin, p[1]-ymin] for p in xy], dtype=np.float32)
    pts = np.round(pts).astype(np.int32)

    mask = np.zeros((ch, cw), dtype=np.uint8)
    if len(pts) >= 3:
        cv2.fillPoly(mask, [pts], 255)
    else:
        mask[:, :] = 255  # 点数不足，退化为矩形

    if crop.ndim == 2:
        crop = cv2.cvtColor(crop, cv2.COLOR_GRAY2BGR)
    b, g, r = cv2.split(crop)
    rgba = cv2.merge([b, g, r, mask])
    return rgba, (xmin, ymin, xmax, ymax)

def safe_write(parent: Path, name: str, img: np.ndarray):
    parent.mkdir(parents=True, exist_ok=True)
    out_path = parent / name
    if out_path.suffix.lower() == ".png":
        cv2.imwrite(str(out_path), img)
    else:
        if img.ndim == 3 and img.shape[2] == 4:  # 去掉 alpha 写 jpg
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        cv2.imwrite(str(out_path), img)

def safe_write_points(parent: Path, name: str, payload: Dict):
    """保存 points 与上下文到 JSON 文件"""
    parent.mkdir(parents=True, exist_ok=True)
    out_path = parent / name
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

def guess_raw_data_dir_from_json(json_path: Path) -> Path:
    """
    从 …/raw_data_sliced_merge/filtered_annotations_*.json 推断 raw_data 目录：
      - v7:   …/<batch>/raw_data_sliced_merge/ -> …/<batch>/raw_data/
      - v4/5/6: …/run_vX/raw_data_sliced_merge/ -> …/run_vX/raw_data/
    """
    sliced = json_path.parent  # raw_data_sliced_merge
    return sliced.parent / "raw_data"

def find_original_image(raw_dir: Path, original_stem: str) -> Optional[Path]:
    """在 raw_data 目录下根据 stem 查找原图；先平铺查找，找不到再递归兜底。"""
    for ext in IMG_EXTS:
        p = raw_dir / f"{original_stem}{ext}"
        if p.exists():
            return p
    pattern = str(raw_dir / "**" / f"{original_stem}*")
    for p in glob.iglob(pattern, recursive=True):
        path = Path(p)
        if path.suffix in IMG_EXTS and path.stem == original_stem:
            return path
    return None

# ========= 主流程 =========
def main():
    OUT_DIR_RECT.mkdir(parents=True, exist_ok=True)
    OUT_DIR_MASK.mkdir(parents=True, exist_ok=True)

    if not JSONL_PATH.exists():
        print(f"[ERROR] 未找到 {JSONL_PATH}")
        return

    roi_cnt = 0
    out_cnt = 0
    missing_img = 0

    with JSONL_PATH.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                rec = json.loads(line)
            except Exception as e:
                print(f"[WARN] 跳过无法解析的行：{e}")
                continue

            uuid = str(rec.get("uuid", "")).lower().strip()
            matches = rec.get("matches", [])
            if not uuid or not matches:
                continue

            for i, m in enumerate(matches):
                json_path = Path(m.get("json_path", ""))
                original_name = str(m.get("original_name", "")).strip()
                xy = to_xy_list(m.get("points"))
                if not original_name or not xy:
                    continue

                raw_dir = guess_raw_data_dir_from_json(json_path)
                big_img_path = find_original_image(raw_dir, original_name)
                if big_img_path is None:
                    print(f"[WARN] 原始图未找到：uuid={uuid}, original_name={original_name}, raw_dir={raw_dir}")
                    missing_img += 1
                    continue

                img = cv2.imread(str(big_img_path), cv2.IMREAD_UNCHANGED)
                if img is None:
                    print(f"[WARN] 无法读取原始图：{big_img_path}")
                    missing_img += 1
                    continue

                # 统一的扁平文件名，避免重名冲突
                # 例：0801_2302_800_uuid_2da2..._roi000_bbox.jpg
                base = f"{big_img_path.stem}_uuid_{uuid}_roi{i:03d}"

                # ===== 正方形裁剪 =====
                if SAVE_BBOX:
                    crop, rect = crop_bbox_square(img, xy)  # 正方形裁剪
                    if crop.size > 0:
                        img_name = f"{base}_bbox.jpg"
                        json_name = f"{base}_bbox.json"
                        safe_write(OUT_DIR_RECT, img_name, crop)

                        xmin, ymin, xmax, ymax = rect
                        points_rel = [[float(x - xmin), float(y - ymin)] for (x, y) in xy]
                        payload = {
                            "type": "bbox",
                            "uuid": uuid,
                            "original_name": original_name,
                            "json_path": str(json_path),
                            "raw_image_path": str(big_img_path),
                            "crop_rect": [int(xmin), int(ymin), int(xmax), int(ymax)],
                            "points_abs": [[float(x), float(y)] for (x, y) in xy],
                            "points_rel": points_rel  # 相对裁剪左上角
                        }
                        safe_write_points(OUT_DIR_RECT, json_name, payload)
                        out_cnt += 1

                # ===== 多边形抠图 =====
                if SAVE_MASK:
                    poly, rect = crop_polygon_mask(img, xy)
                    if poly.size > 0:
                        img_name = f"{base}_mask.png"
                        json_name = f"{base}_mask.json"
                        safe_write(OUT_DIR_MASK, img_name, poly)

                        xmin, ymin, xmax, ymax = rect
                        points_rel = [[float(x - xmin), float(y - ymin)] for (x, y) in xy]
                        payload = {
                            "type": "mask",
                            "uuid": uuid,
                            "original_name": original_name,
                            "json_path": str(json_path),
                            "raw_image_path": str(big_img_path),
                            "crop_rect": [int(xmin), int(ymin), int(xmax), int(ymax)],
                            "points_abs": [[float(x), float(y)] for (x, y) in xy],
                            "points_rel": points_rel  # 相对裁剪左上角
                        }
                        safe_write_points(OUT_DIR_MASK, json_name, payload)
                        out_cnt += 1

                roi_cnt += 1

    print(f"[DONE] 共处理 ROI：{roi_cnt}，输出文件：{out_cnt}。")
    print(f"[INFO] 原图缺失/不可读：{missing_img} 次。")
    print(f"[OUT ] 正方形裁剪：{OUT_DIR_RECT.resolve()}")
    print(f"[OUT ] 多边形抠图：{OUT_DIR_MASK.resolve()}")

if __name__ == "__main__":
    main()


[DONE] 共处理 ROI：849，输出文件：1698。
[INFO] 原图缺失/不可读：0 次。
[OUT ] 正方形裁剪：/workspace/models/runs_yolov11_det/crops_by_points/rect
[OUT ] 多边形抠图：/workspace/models/runs_yolov11_det/crops_by_points/mask


# 图片放大

In [63]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
import cv2
import glob
import numpy as np
from pathlib import Path
from typing import List, Tuple, Any, Dict, Optional

# ========== 配置（请改这里） ==========
IN_DIR      = Path("/workspace/models/runs_yolov11_det/crops_by_points/swd/mask")  # ← 你的“已导出裁剪+JSON”的目录（不回源头）
OUT_RESIZED = Path("/workspace/models/runs_yolov11_det/crops_by_points/swd/mask/resized_640_lb")         # ← 640×640(等比+补边) 的图片和更新后的 JSON
OUT_VIS     = Path("/workspace/models/runs_yolov11_det/crops_by_points/swd/mask/resized_640_lb_vis")     # ← 叠加 Polygon 的可视化图片
TARGET_SIZE = 640

IMG_EXTS = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]

# ========== 工具 ==========
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def is_image(p: Path) -> bool:
    return p.suffix in IMG_EXTS

def paired_json_for(img_path: Path) -> Path:
    return img_path.with_suffix(".json")

def _normalize_points_list(points: Any) -> Optional[List[List[float]]]:
    """兼容 [[x,y], ...] 或 [{'x':..,'y':..}, ...] -> [[x,y], ...]"""
    if points is None:
        return None
    out = []
    if isinstance(points, (list, tuple)):
        for pt in points:
            if isinstance(pt, (list, tuple)) and len(pt) >= 2:
                out.append([float(pt[0]), float(pt[1])])
            elif isinstance(pt, dict) and ("x" in pt) and ("y" in pt):
                out.append([float(pt["x"]), float(pt["y"])])
    return out if out else None

def load_rel_points_for_image(json_path: Path) -> Optional[List[List[float]]]:
    """
    读取与“当前裁剪图”同名的 JSON，返回该图坐标系下的点（未放大前的相对坐标）。
    优先 points_rel；否则尝试 points_abs + crop_rect；最后尝试 points。
    """
    if not json_path.exists():
        return None
    with json_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    if isinstance(data, dict) and "points_rel" in data:
        return _normalize_points_list(data["points_rel"])

    if isinstance(data, dict) and ("points_abs" in data) and ("crop_rect" in data):
        pts_abs = _normalize_points_list(data["points_abs"])
        rect = data.get("crop_rect", None)
        if isinstance(rect, (list, tuple)) and len(rect) >= 4 and pts_abs:
            xmin, ymin = float(rect[0]), float(rect[1])
            return [[x - xmin, y - ymin] for (x, y) in pts_abs]

    if isinstance(data, dict) and "points" in data:
        return _normalize_points_list(data["points"])

    return None

def letterbox_resize(img: np.ndarray, out_size: int = 640,
                     pad_value_bgr=(0,0,0)) -> Tuple[np.ndarray, float, Tuple[int, int, int, int]]:
    """
    等比缩放 + 补边到 out_size x out_size：
    返回：resized_img, scale, pads(left, top, right, bottom)
    - 若有 alpha，保持 alpha，并对 alpha 也做补边（alpha 补 0 = 全透明）。
    """
    h, w = img.shape[:2]
    if h == 0 or w == 0:
        return img, 1.0, (0, 0, 0, 0)

    scale = min(out_size / float(w), out_size / float(h))
    new_w = int(round(w * scale))
    new_h = int(round(h * scale))

    interp = cv2.INTER_CUBIC if scale > 1.0 else cv2.INTER_AREA

    if img.ndim == 2:
        resized = cv2.resize(img, (new_w, new_h), interpolation=interp)
        canvas = np.full((out_size, out_size), pad_value_bgr[0], dtype=resized.dtype)
        left = (out_size - new_w) // 2
        top  = (out_size - new_h) // 2
        canvas[top:top+new_h, left:left+new_w] = resized
        right = out_size - left - new_w
        bottom = out_size - top - new_h
        return canvas, scale, (left, top, right, bottom)

    if img.shape[2] == 4:
        b, g, r, a = cv2.split(img)
        bgr = cv2.merge([b, g, r])
        bgr_resized = cv2.resize(bgr, (new_w, new_h), interpolation=interp)
        a_resized   = cv2.resize(a,   (new_w, new_h), interpolation=interp)

        canvas_bgr = np.full((out_size, out_size, 3), pad_value_bgr, dtype=bgr_resized.dtype)
        canvas_a   = np.zeros((out_size, out_size), dtype=a_resized.dtype)  # 透明补边

        left = (out_size - new_w) // 2
        top  = (out_size - new_h) // 2
        canvas_bgr[top:top+new_h, left:left+new_w] = bgr_resized
        canvas_a  [top:top+new_h, left:left+new_w] = a_resized

        right = out_size - left - new_w
        bottom= out_size - top - new_h

        b2, g2, r2 = cv2.split(canvas_bgr)
        canvas_rgba = cv2.merge([b2, g2, r2, canvas_a])
        return canvas_rgba, scale, (left, top, right, bottom)

    # 普通 BGR
    resized = cv2.resize(img, (new_w, new_h), interpolation=interp)
    canvas = np.full((out_size, out_size, 3), pad_value_bgr, dtype=resized.dtype)
    left = (out_size - new_w) // 2
    top  = (out_size - new_h) // 2
    canvas[top:top+new_h, left:left+new_w] = resized
    right = out_size - left - new_w
    bottom= out_size - top - new_h
    return canvas, scale, (left, top, right, bottom)

def map_points_scale_pad(points: Optional[List[List[float]]], scale: float, pad: Tuple[int,int,int,int]) -> Optional[List[List[float]]]:
    """点位按 scale 缩放后，再加上 left/top 的补边偏移。"""
    if not points:
        return None
    left, top, _, _ = pad
    out = []
    for x, y in points:
        out.append([x * scale + left, y * scale + top])
    return out

def save_image(out_path: Path, img: np.ndarray):
    ensure_dir(out_path.parent)
    if out_path.suffix.lower() == ".png":
        cv2.imwrite(str(out_path), img)
    else:
        if img.ndim == 3 and img.shape[2] == 4:
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        cv2.imwrite(str(out_path), img)

def draw_polygon_on_image(img: np.ndarray,
                          pts: List[List[float]],
                          fill_alpha: float = 0.25,
                          edge_thickness: int = 2) -> np.ndarray:
    """在 img 上绘制 polygon（半透明填充+边界+点）"""
    if img is None or img.size == 0 or not pts:
        return img
    canvas = img
    if canvas.ndim == 3 and canvas.shape[2] == 4:
        canvas = cv2.cvtColor(canvas, cv2.COLOR_BGRA2BGR)

    overlay = canvas.copy()
    np_pts = np.array(pts, dtype=np.float32)
    np_pts = np.round(np_pts).astype(np.int32)

    color_fill = (255, 0, 0)   # BGR
    color_edge = (0, 0, 255)
    color_dot  = (0, 255, 255)

    if len(np_pts) >= 3:
        cv2.fillPoly(overlay, [np_pts], color_fill)
        canvas = cv2.addWeighted(overlay, fill_alpha, canvas, 1 - fill_alpha, 0)
    if len(np_pts) >= 2:
        cv2.polylines(canvas, [np_pts], isClosed=True, color=color_edge, thickness=edge_thickness)
    for p in np_pts:
        cv2.circle(canvas, (int(p[0]), int(p[1])), radius=3, color=color_dot, thickness=-1)

    return canvas

# ========== JSON 写入 ==========
def update_json_scaled_letterbox(in_json: Path, out_json: Path,
                                 scale: float, pad: Tuple[int,int,int,int],
                                 in_w: int, in_h: int, out_w: int, out_h: int,
                                 scaled_points_rel: Optional[List[List[float]]]) -> None:
    with in_json.open("r", encoding="utf-8") as f:
        data = json.load(f)

    if scaled_points_rel is not None:
        data["points_rel_resized"] = [[float(x), float(y)] for (x, y) in scaled_points_rel]

    meta = data.get("meta", {})
    meta.update({
        "method": "letterbox",
        "resized_from": [int(in_w), int(in_h)],
        "resized_to": [int(out_w), int(out_h)],
        "scale": float(scale),
        "pad": {
            "left": int(pad[0]),
            "top": int(pad[1]),
            "right": int(pad[2]),
            "bottom": int(pad[3])
        }
    })
    data["meta"] = meta

    ensure_dir(out_json.parent)
    with out_json.open("w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# ========== 主处理 ==========
def process_one(img_path: Path):
    img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
    if img is None:
        print(f"[WARN] 读取失败：{img_path}")
        return

    h, w = img.shape[:2]
    in_json = paired_json_for(img_path)
    rel_points = load_rel_points_for_image(in_json)  # 未放大前的相对坐标（当前图坐标系）

    # 等比缩放 + 补边
    lb_img, scale, pad = letterbox_resize(img, out_size=TARGET_SIZE, pad_value_bgr=(114,114,114))

    # 点位同步：先 *scale，再 +left/top
    scaled_rel_points = map_points_scale_pad(rel_points, scale, pad)

    # 输出路径
    rel = img_path.relative_to(IN_DIR)
    out_img  = OUT_RESIZED / rel
    out_vis  = OUT_VIS / rel.with_suffix(".jpg")     # 可视化统一用 JPG
    out_json = OUT_RESIZED / rel.with_suffix(".json")

    # 写图
    save_image(out_img, lb_img)

    # 写 JSON
    if in_json.exists():
        update_json_scaled_letterbox(
            in_json, out_json,
            scale=scale, pad=pad,
            in_w=w, in_h=h, out_w=TARGET_SIZE, out_h=TARGET_SIZE,
            scaled_points_rel=scaled_rel_points
        )

    # 可视化
    if scaled_rel_points is not None:
        vis = draw_polygon_on_image(lb_img.copy(), scaled_rel_points,
                                    fill_alpha=0.25, edge_thickness=2)
    else:
        vis = lb_img.copy()

    save_image(out_vis, vis)

    print(f"[OK] {img_path} -> {out_img.name}, {out_vis.name} (scale={scale:.4f}, pad={pad})")

def main():
    if not IN_DIR.exists():
        print(f"[ERROR] 输入目录不存在：{IN_DIR}")
        return
    ensure_dir(OUT_RESIZED)
    ensure_dir(OUT_VIS)

    patterns = [str(IN_DIR / "**" / f"*{ext}") for ext in IMG_EXTS]
    files = []
    for pat in patterns:
        files.extend(glob.glob(pat, recursive=True))
    img_files = [Path(p) for p in files]

    if not img_files:
        print(f"[INFO] 未找到图片：{IN_DIR}")
        return

    cnt = 0
    for p in img_files:
        try:
            process_one(p)
            cnt += 1
        except Exception as e:
            print(f"[ERR ] 处理失败：{p} -> {e}")

    print(f"[DONE] 共处理图片：{cnt}")
    print(f"[OUT ] 放大图 & 更新 JSON：{OUT_RESIZED.resolve()}")
    print(f"[OUT ] 可视化图：{OUT_VIS.resolve()}")

if __name__ == "__main__":
    main()


[OK] /workspace/models/runs_yolov11_det/crops_by_points/swd/mask/0607_0731_700_uuid_35af4292-800d-4e97-91e4-eca5e5229ec9_roi000_mask.png -> 0607_0731_700_uuid_35af4292-800d-4e97-91e4-eca5e5229ec9_roi000_mask.png, 0607_0731_700_uuid_35af4292-800d-4e97-91e4-eca5e5229ec9_roi000_mask.jpg (scale=10.8475, pad=(0, 16, 0, 17))
[OK] /workspace/models/runs_yolov11_det/crops_by_points/swd/mask/0607_0731_700_uuid_ce655b7d-7c8b-4a1e-a863-32a000d82d0d_roi000_mask.png -> 0607_0731_700_uuid_ce655b7d-7c8b-4a1e-a863-32a000d82d0d_roi000_mask.png, 0607_0731_700_uuid_ce655b7d-7c8b-4a1e-a863-32a000d82d0d_roi000_mask.jpg (scale=11.4286, pad=(17, 0, 17, 0))
[OK] /workspace/models/runs_yolov11_det/crops_by_points/swd/mask/0607_0801_700_uuid_50a739e6-61d0-4b8b-aaeb-a692c56aa56e_roi000_mask.png -> 0607_0801_700_uuid_50a739e6-61d0-4b8b-aaeb-a692c56aa56e_roi000_mask.png, 0607_0801_700_uuid_50a739e6-61d0-4b8b-aaeb-a692c56aa56e_roi000_mask.jpg (scale=11.6364, pad=(0, 17, 0, 18))
[OK] /workspace/models/runs_yolov11_d

# 背景填充 -- 黑色

In [65]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import cv2
import glob
import numpy as np
from pathlib import Path

# ========== 配置 ==========
# IN_DIR  = Path("/workspace/models/runs_yolov11_det/crops_by_points/swd/mask/resized_640_lb")   # 输入目录
IN_DIR  = Path("/workspace/models/runs_yolov11_det/crops_by_points/mayswd/mask/resized_640_lb/")   # 输入目录
# OUT_DIR = Path("/workspace/models/runs_yolov11_det/crops_by_points/swd/mask/resized_640_lb_blackbg")  # 输出目录
OUT_DIR = Path("/workspace/models/runs_yolov11_det/crops_by_points/mayswd/mask/resized_640_lb_blackbg")  # 输出目录

IMG_EXTS = [".png", ".PNG"]

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def process_one(img_path: Path, out_dir: Path):
    img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
    if img is None:
        print(f"[WARN] 无法读取：{img_path}")
        return

    if img.shape[2] == 4:  # 带 alpha 通道
        b, g, r, a = cv2.split(img)
        # alpha==0 的区域 → 填充黑色
        mask = (a == 0)
        b[mask] = 0
        g[mask] = 0
        r[mask] = 0
        merged = cv2.merge([b, g, r])  # 输出为 BGR 三通道
    else:
        merged = img  # 没有透明通道就直接拷贝

    out_path = out_dir / img_path.relative_to(IN_DIR)
    ensure_dir(out_path.parent)
    cv2.imwrite(str(out_path.with_suffix(".jpg")), merged)  # 存 JPG（无透明）
    print(f"[OK] {img_path} -> {out_path.with_suffix('.jpg')}")

def main():
    ensure_dir(OUT_DIR)
    files = []
    for ext in IMG_EXTS:
        files.extend(glob.glob(str(IN_DIR / f"**/*{ext}"), recursive=True))

    if not files:
        print(f"[INFO] 没有找到 PNG：{IN_DIR}")
        return

    for f in files:
        process_one(Path(f), OUT_DIR)

    print(f"[DONE] 已处理 {len(files)} 张图片。输出目录：{OUT_DIR.resolve()}")

if __name__ == "__main__":
    main()


[OK] /workspace/models/runs_yolov11_det/crops_by_points/mayswd/mask/resized_640_lb/0718_0604_580_uuid_d1d94c90-a467-47d0-b016-e6ebfa103e8d_roi000_mask.png -> /workspace/models/runs_yolov11_det/crops_by_points/mayswd/mask/resized_640_lb_blackbg/0718_0604_580_uuid_d1d94c90-a467-47d0-b016-e6ebfa103e8d_roi000_mask.jpg
[OK] /workspace/models/runs_yolov11_det/crops_by_points/mayswd/mask/resized_640_lb/0718_0607_640_uuid_966802cc-f544-4fae-a1d4-0db8ea6d2f51_roi000_mask.png -> /workspace/models/runs_yolov11_det/crops_by_points/mayswd/mask/resized_640_lb_blackbg/0718_0607_640_uuid_966802cc-f544-4fae-a1d4-0db8ea6d2f51_roi000_mask.jpg
[OK] /workspace/models/runs_yolov11_det/crops_by_points/mayswd/mask/resized_640_lb/0718_0634_580_uuid_05727c5d-7df8-45ae-a2be-e3e13d2c41c1_roi000_mask.png -> /workspace/models/runs_yolov11_det/crops_by_points/mayswd/mask/resized_640_lb_blackbg/0718_0634_580_uuid_05727c5d-7df8-45ae-a2be-e3e13d2c41c1_roi000_mask.jpg
[OK] /workspace/models/runs_yolov11_det/crops_by_poi

# 背景高斯模糊

In [85]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
import cv2
import glob
import numpy as np
from pathlib import Path
from typing import List, Tuple, Any, Optional, Union

# ========== 配置（请按需修改） ==========
# IN_DIR      = Path("/workspace/models/runs_yolov11_det/crops_by_points/mayswd/rect/resized_640_lb")  # 输入：jpg + json
# OUT_BLUR    = Path("/workspace/models/runs_yolov11_det/crops_by_points/mayswd/rect/resized_640_lb_blur_outside")  # 新文件夹1
# OUT_VIS     = Path("/workspace/models/runs_yolov11_det/crops_by_points/mayswd/rect/resized_640_lb_blur_outside_vis")  # 新文件夹2

IN_DIR      = Path("/workspace/models/runs_yolov11_det/crops_by_points/swd/rect/resized_640_lb")  # 输入：jpg + json
OUT_BLUR    = Path("/workspace/models/runs_yolov11_det/crops_by_points/swd/rect/resized_640_lb_blur_outside")  # 新文件夹1
OUT_VIS     = Path("/workspace/models/runs_yolov11_det/crops_by_points/swd/rect/resized_640_lb_blur_outside_vis")  # 新文件夹2

# 模糊强度（内核需奇数）；sigmaX 留 0 让 OpenCV 自算
GAUSSIAN_KERNEL = (101, 101)
GAUSSIAN_SIGMA  = 0

# 画边颜色/厚度与填充透明度（BGR）
EDGE_COLOR   = (0, 0, 255)   # 红色边
EDGE_THICK   = 2
FILL_COLOR   = (255, 0, 0)   # 蓝色内部半透明
FILL_ALPHA   = 0.20          # 0~1；设 0 关闭填充效果

IMG_EXTS = [".jpg", ".jpeg", ".JPG", ".JPEG"]

# ========== 工具 ==========
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def paired_json_for(img_path: Path) -> Path:
    return img_path.with_suffix(".json")

def _as_point_list(x: Any) -> Optional[List[List[float]]]:
    """
    把 [[x,y],...] 或 [{'x':..,'y':..},...] 的点集转成 [[x,y],...]
    若是“多个多边形”的结构（例如 [[[...],[...],...], [[...],...]]），则原样返回嵌套结构
    """
    if x is None:
        return None
    # 检测是否为多多边形（list 的第一层元素也是 list，并且其第一项仍是 list/dict）
    if isinstance(x, list) and len(x) > 0 and isinstance(x[0], list) and len(x[0]) > 0 and isinstance(x[0][0], (list, dict, float, int)):
        # 可能是单个多边形或多多边形
        # 判断：若 x[0][0] 还是标量，则是单多边形
        if isinstance(x[0][0], (int, float)) or (isinstance(x[0][0], list) and len(x[0][0]) == 2):
            # 单个多边形
            return _normalize_one_polygon(x)
        else:
            # 多个多边形：对每个子项递归归一化
            polys = []
            for poly in x:
                polys.append(_normalize_one_polygon(poly))
            return polys
    # dict/list 扁平点集
    return _normalize_one_polygon(x)

def _normalize_one_polygon(points: Any) -> Optional[List[List[float]]]:
    out = []
    if isinstance(points, list):
        for pt in points:
            if isinstance(pt, (list, tuple)) and len(pt) >= 2:
                out.append([float(pt[0]), float(pt[1])])
            elif isinstance(pt, dict) and ("x" in pt) and ("y" in pt):
                out.append([float(pt["x"]), float(pt["y"])])
    return out if out else None

def load_polygon_points_for_image(json_path: Path, img_shape: Tuple[int,int]) -> Optional[Union[List[List[float]], List[List[List[float]]]]]:
    """
    从 JSON 中得到“当前图坐标系”的多边形点：
      优先：points_rel_resized
      其次：points_rel + meta.scale/meta.pad（在 640 letterbox 下映射）
      再次：points（假定即当前坐标）
    支持“单多边形”或“多多边形”（返回同结构）。
    """
    if not json_path.exists():
        return None
    with json_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    # 1) 直接使用 points_rel_resized
    if "points_rel_resized" in data:
        pts = _as_point_list(data["points_rel_resized"])
        if pts: return pts

    # 2) 由 points_rel + meta.scale + meta.pad 推算（适用于 letterbox 到 640x640）
    if "points_rel" in data and "meta" in data and isinstance(data["meta"], dict):
        pts_rel = _as_point_list(data["points_rel"])
        scale = data["meta"].get("scale", None)
        pad   = data["meta"].get("pad", None)

        if pts_rel and scale is not None and pad is not None:
            # scale 既可能是 float，也可能是 {"sx":..,"sy":..}；letterbox 版本是 float
            if isinstance(scale, dict):
                sx = float(scale.get("sx", 1.0))
                sy = float(scale.get("sy", 1.0))
            else:
                sx = sy = float(scale)

            left = int(pad.get("left", 0)) if isinstance(pad, dict) else 0
            top  = int(pad.get("top", 0))  if isinstance(pad, dict) else 0

            def map_pts(one_poly):
                return [[x * sx + left, y * sy + top] for (x, y) in one_poly]

            if isinstance(pts_rel[0][0], list):  # 多多边形
                return [map_pts(poly) for poly in pts_rel]
            else:  # 单个多边形
                return map_pts(pts_rel)

    # 3) 兜底：points（假设它本来就是当前图坐标）
    if "points" in data:
        pts = _as_point_list(data["points"])
        if pts: return pts

    return None

def polygons_to_mask(polys: Union[List[List[float]], List[List[List[float]]]], h: int, w: int) -> np.ndarray:
    """
    根据单/多多边形生成 mask（uint8, 0/255）。点数<3时退化为外接矩形。
    """
    mask = np.zeros((h, w), dtype=np.uint8)

    def draw_one(poly):
        if poly is None or len(poly) == 0:
            return
        pts = np.array(poly, dtype=np.float32)
        pts = np.round(pts).astype(np.int32)
        if len(pts) >= 3:
            cv2.fillPoly(mask, [pts], 255)
        else:
            # 退化为外接矩形
            xs, ys = pts[:, 0], pts[:, 1]
            xmin, xmax = int(np.floor(xs.min())), int(np.ceil(xs.max()))
            ymin, ymax = int(np.floor(ys.min())), int(np.ceil(ys.max()))
            xmin = max(0, min(xmin, w-1)); xmax = max(0, min(xmax, w-1))
            ymin = max(0, min(ymin, h-1)); ymax = max(0, min(ymax, h-1))
            if xmax < xmin: xmax = xmin
            if ymax < ymin: ymax = ymin
            mask[ymin:ymax+1, xmin:xmax+1] = 255

    # 多多边形
    if isinstance(polys, list) and len(polys) > 0 and isinstance(polys[0], list) and len(polys[0]) > 0 and isinstance(polys[0][0], list):
        for poly in polys:
            draw_one(poly)
    else:
        draw_one(polys)

    return mask

def blur_outside_polygon(img: np.ndarray, mask: np.ndarray,
                         ksize=(31,31), sigmaX=0) -> np.ndarray:
    """
    多边形外高斯模糊：mask 内保持清晰，外部模糊。
    """
    # 统一到 BGR（jpeg 不会有 alpha）
    base = img.copy()
    if base.ndim == 3 and base.shape[2] == 4:
        base = cv2.cvtColor(base, cv2.COLOR_BGRA2BGR)

    blurred = cv2.GaussianBlur(base, ksize, sigmaX)
    m3 = cv2.merge([mask, mask, mask])
    inv = cv2.bitwise_not(mask)
    inv3 = cv2.merge([inv, inv, inv])

    keep = cv2.bitwise_and(base, m3)
    blur = cv2.bitwise_and(blurred, inv3)
    out  = cv2.add(keep, blur)
    return out

def draw_polygon_on_image(img: np.ndarray,
                          polys: Union[List[List[float]], List[List[List[float]]]],
                          edge_color=(0,0,255), edge_thick=2,
                          fill_color=(255,0,0), fill_alpha=0.2) -> np.ndarray:
    """
    在图上描边/可选半透明填充，支持单/多多边形。
    """
    canvas = img.copy()
    if canvas.ndim == 3 and canvas.shape[2] == 4:
        canvas = cv2.cvtColor(canvas, cv2.COLOR_BGRA2BGR)

    def draw_one(poly):
        pts = np.array(poly, dtype=np.float32)
        pts = np.round(pts).astype(np.int32)
        if len(pts) >= 3 and fill_alpha > 0:
            overlay = canvas.copy()
            cv2.fillPoly(overlay, [pts], fill_color)
            canvas[:] = cv2.addWeighted(overlay, fill_alpha, canvas, 1 - fill_alpha, 0)
        if len(pts) >= 2:
            cv2.polylines(canvas, [pts], isClosed=True, color=edge_color, thickness=edge_thick)

    if isinstance(polys, list) and len(polys) > 0 and isinstance(polys[0], list) and len(polys[0]) > 0 and isinstance(polys[0][0], list):
        for poly in polys:
            draw_one(poly)
    else:
        draw_one(polys)

    return canvas

def process_one(img_path: Path):
    img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
    if img is None:
        print(f"[WARN] 无法读取：{img_path}")
        return

    h, w = img.shape[:2]
    json_path = paired_json_for(img_path)
    polys = load_polygon_points_for_image(json_path, (h, w))
    if polys is None:
        print(f"[WARN] 未找到可用的 points：{json_path}")
        return

    # 生成 mask 并在外部模糊
    mask = polygons_to_mask(polys, h, w)
    blur_img = blur_outside_polygon(img, mask, ksize=GAUSSIAN_KERNEL, sigmaX=GAUSSIAN_SIGMA)

    # 路径 & 保存 新文件夹1（仅模糊）
    rel = img_path.relative_to(IN_DIR)
    out1 = OUT_BLUR / rel
    ensure_dir(out1.parent)
    cv2.imwrite(str(out1), blur_img)

    # 新文件夹2（在模糊结果上描边/填充）
    vis = draw_polygon_on_image(blur_img, polys,
                                edge_color=EDGE_COLOR, edge_thick=EDGE_THICK,
                                fill_color=FILL_COLOR, fill_alpha=FILL_ALPHA)
    out2 = OUT_VIS / rel
    ensure_dir(out2.parent)
    cv2.imwrite(str(out2), vis)

    print(f"[OK] {img_path} -> {out1} & {out2}")

def main():
    ensure_dir(OUT_BLUR)
    ensure_dir(OUT_VIS)

    files = []
    for ext in IMG_EXTS:
        files.extend(glob.glob(str(IN_DIR / f"**/*{ext}"), recursive=True))
    img_files = [Path(p) for p in files]

    if not img_files:
        print(f"[INFO] 未找到图片：{IN_DIR}")
        return

    cnt = 0
    for p in img_files:
        try:
            process_one(p)
            cnt += 1
        except Exception as e:
            print(f"[ERR ] 处理失败：{p} -> {e}")

    print(f"[DONE] 共处理图片：{cnt}")
    print(f"[OUT ] 模糊外部：{OUT_BLUR.resolve()}")
    print(f"[OUT ] 模糊外部+描边：{OUT_VIS.resolve()}")

if __name__ == "__main__":
    main()


[OK] /workspace/models/runs_yolov11_det/crops_by_points/swd/rect/resized_640_lb/0607_0731_700_uuid_35af4292-800d-4e97-91e4-eca5e5229ec9_roi000_bbox.jpg -> /workspace/models/runs_yolov11_det/crops_by_points/swd/rect/resized_640_lb_blur_outside/0607_0731_700_uuid_35af4292-800d-4e97-91e4-eca5e5229ec9_roi000_bbox.jpg & /workspace/models/runs_yolov11_det/crops_by_points/swd/rect/resized_640_lb_blur_outside_vis/0607_0731_700_uuid_35af4292-800d-4e97-91e4-eca5e5229ec9_roi000_bbox.jpg
[OK] /workspace/models/runs_yolov11_det/crops_by_points/swd/rect/resized_640_lb/0607_0731_700_uuid_ce655b7d-7c8b-4a1e-a863-32a000d82d0d_roi000_bbox.jpg -> /workspace/models/runs_yolov11_det/crops_by_points/swd/rect/resized_640_lb_blur_outside/0607_0731_700_uuid_ce655b7d-7c8b-4a1e-a863-32a000d82d0d_roi000_bbox.jpg & /workspace/models/runs_yolov11_det/crops_by_points/swd/rect/resized_640_lb_blur_outside_vis/0607_0731_700_uuid_ce655b7d-7c8b-4a1e-a863-32a000d82d0d_roi000_bbox.jpg
[OK] /workspace/models/runs_yolov11_de